In [6]:
import sys
import itertools
import argparse

from treys import Card
from treys import Evaluator
from treys import Deck

from rules import *
from utils import lget

POKER_HAND_EVALUATOR = Evaluator()


def isTypeOfCard(card, desiredCards):
    for desiredCard in desiredCards:
        if desiredCard in card:
            desiredCards.remove(desiredCard)
            return True
    return False

def simulatePokerSolitaire(desiredCards):
    """Simulates the game of playPoker.
    Args:
        cardsDesired: the cards that the
        player specified to want to draw
    Returns:
        playerCards: the cards drawn by
        the player
        communityCards: the cards drawn by
        the community
    """
    playerCards, communityCards, deck = [], [], Deck()
    while len(playerCards) < 5 or len(communityCards) < 5:
        card = deck.draw(1)
        if len(playerCards) < 5:
            stringCardDrawn = Card.print_pretty_card(card)
            if isTypeOfCard(stringCardDrawn, desiredCards):
                playerCards.append(card)
        communityCards.append(card)
    
    return playerCards, communityCards

def findStrongestCombinationAndItsStrength(totalCombinations):
    combination = totalCombinations[0]
    handStrength = POKER_HAND_EVALUATOR.evaluate(list(combination), [])
    for index in range(0, len(totalCombinations)):
        tempHandStrength = POKER_HAND_EVALUATOR.evaluate(
            list(totalCombinations[index]), []
        )
        if tempHandStrength <= handStrength:
            handStrength = tempHandStrength
            combination = totalCombinations[index]

    return combination, handStrength


def evaluateBetterHand(playerCards, communityCards):
    """Evaluates the stronger hand between the player's hand
    and the community's strongest 5 card combination
    Args:
        playerCards --> player's 5 cards
        communityCards --> community's collected cards
    Returns:
        boolean --> if playerHand is lower (stronger)
        bestCommunityCombination --> strongest possible hand that
        the community has
    """
    playerHandStrength = POKER_HAND_EVALUATOR.evaluate(playerCards, [])
    totalCombinations = list(itertools.combinations(communityCards, 5))

    (
        communityStrongestCombo,
        communityHandStrength,
    ) = findStrongestCombinationAndItsStrength(totalCombinations)

    return (
        communityHandStrength >= playerHandStrength,
        communityStrongestCombo,
    )


def playPoker(enterHand):
    enterHandInSymbols = [Card.print_pretty_card(Card.new(card)) if len(card) > 1 else card for card in enterHand]
    print(enterHandInSymbols)
    playerCards, communityCards = simulatePokerSolitaire(enterHandInSymbols)
    playerWin, bestCommunityCombination = evaluateBetterHand(
        playerCards, communityCards
    )

    return (
        playerWin,
        Card.print_pretty_cards(playerCards),
        Card.print_pretty_cards(communityCards),
        Card.print_pretty_cards(bestCommunityCombination),
    )

In [7]:
enterHand = ['4d', '3', 'As', '2d', '5c']
playPoker(enterHand)

['[4♦]', '3', '[A♠]', '[2♦]', '[5♣]']
[8423187, 134253349, 33573149, 164099, 134224677, 270853, 268471337, 2114829, 8394515, 4212241, 1053707, 8406803, 2102541, 33564957, 81922, 67119647, 98306, 2106637, 557831, 33560861, 67115551, 268446761, 8398611, 69634, 134236965, 67144223, 73730, 268454953, 541447, 16783383, 533255, 268442665, 295429, 1065995, 4204049, 67127839, 16787479, 33589533, 134228773, 4199953, 4228625, 279045]


(False,
 ' [3♣],[2♦],[5♣],[A♠],[4♦] ',
 ' [9♣],[K♣],[J♦],[3♣],[K♠],[4♥],[A♣],[7♦],[9♠],[8♦],[6♠],[9♦],[7♠],[J♥],[2♦],[Q♥],[2♣],[7♥],[5♣],[J♠],[Q♠],[A♥],[9♥],[2♠],[K♦],[Q♣],[2♥],[A♦],[5♦],[T♠],[5♥],[A♠],[4♣],[6♦],[8♥],[Q♦],[T♥],[J♣],[K♥],[8♠],[8♣],[4♦] ',
 ' [J♥],[Q♥],[A♥],[T♥],[K♥] ')

In [ ]:
enterHand = ['4d', '3s', 'As', '2d', '5c']
x = [Card.print_pretty_card(Card.new(x)) if len(x) > 1 else card for x in enterHand]

In [ ]:
cards = []
for card in enterHand:
    if len(card) > 1:
        cards.append(Card.print_pretty_card(Card.new(card)))
    else:
        cards.append(card)

In [ ]:
cards

In [ ]:
print('3' in '[3♠]')

In [4]:
x = itertools.combinations([98306, 33589533, 268446761, 268454953, 134224677, 1082379, 266757, 16795671, 8398611], 5)

In [ ]:
for x in x:
    print(x)